In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from keras import callbacks

In [2]:
batch_size = 32
img_height = 32
img_width = 32

IMG_SIZE = 32

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train/255
x_test = x_test/255

num_classes = np.size(np.unique(y_train))

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170498071/170498071 [==============================] - 4s 0us/step


In [4]:
base_model = keras.applications.MobileNet(
                        include_top=False, # Exclude ImageNet classifier at the top
                        weights='imagenet',
                        input_shape=(IMG_SIZE, IMG_SIZE, 3)
                        )

# Freeze base_model
base_model.trainable = False

# Setup inputs based on input image shape
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

data_augmentation = keras.Sequential(
                [layers.RandomFlip('horizontal')])

x = data_augmentation(inputs)

# Apply specific pre-processing function for ResNet v2
x = keras.applications.mobilenet.preprocess_input(x)

# Keep base model batch normalization layers in inference mode (instead of training mode)
x = base_model(x, training=False)

# Rebuild top layers
x = layers.GlobalAveragePooling2D()(x) # Average pooling operation
x = layers.BatchNormalization()(x) # Introduce batch norm
x = layers.Dropout(0.2)(x)  # Regularize with dropout

# Flattening to final layer - Dense classifier with 37 units (multi-class classification)
outputs = layers.Dense(num_classes, activation='softmax')(x)

# Instantiate final Keras model with updated inputs and outputs
model = keras.Model(inputs, outputs)

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()]
              )

earlystopping = callbacks.EarlyStopping(monitor='val_loss',
                                        mode='min',
                                        patience=5,
                                        restore_best_weights=True)

EPOCHS = 25

history = model.fit(x_train, y_train,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    verbose=1,
                    callbacks =[earlystopping])

Epoch 1/25
1563/1563 [==============================] - 22s 10ms/step - loss: 2.1815 - categorical_accuracy: 0.2054 - val_loss: 2.1202 - val_categorical_accuracy: 0.2447
Epoch 2/25
1563/1563 [==============================] - 15s 10ms/step - loss: 2.1576 - categorical_accuracy: 0.2180 - val_loss: 2.1168 - val_categorical_accuracy: 0.2470
Epoch 3/25
1563/1563 [==============================] - 12s 7ms/step - loss: 2.1566 - categorical_accuracy: 0.2176 - val_loss: 2.1141 - val_categorical_accuracy: 0.2458
Epoch 4/25
1563/1563 [==============================] - 13s 8ms/step - loss: 2.1542 - categorical_accuracy: 0.2162 - val_loss: 2.1149 - val_categorical_accuracy: 0.2502
Epoch 5/25
1563/1563 [==============================] - 12s 8ms/step - loss: 2.1555 - categorical_accuracy: 0.2174 - val_loss: 2.1130 - val_categorical_accuracy: 0.2548
Epoch 6/25
1563/1563 [==============================] - 12s 8ms/step - loss: 2.1528 - categorical_accuracy: 0.2200 - val_loss: 2.1140 - val_categorical_a

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history['categorical_accuracy'])
    plt.plot(hist.history['val_categorical_accuracy'])
    plt.title('Categorical accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

plot_hist(history)